# 1. Import Data

In [ ]:
import obspy
from obspy import read
import numpy
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import scipy.signal as signal


In [ ]:
# get data:
# scp yrossi@nagler:../../import/roth-data/fbernauer/GIOTTO_ROBUST/mseed/XX.GIO11..DNX.D.2023.174 Documents/Work/Skience25/2_Data/GIOTTO

day = '174'
station = 'GIO10'

root2data = './data'
acc_orig = read('%s/XX.%s..DNX.D.2023.%s' % (root2data,station,day))
acc_orig += read('%s/XX.%s..DNY.D.2023.%s' % (root2data,station,day))
acc_orig += read('%s/XX.%s..DNZ.D.2023.%s' % (root2data,station,day))

rot_rate_orig = read('%s/XX.%s..DJX.D.2023.%s' % (root2data,station,day))
rot_rate_orig += read('%s/XX.%s..DJY.D.2023.%s' % (root2data,station,day))
rot_rate_orig += read('%s/XX.%s..DJZ.D.2023.%s' % (root2data,station,day))


print(acc_orig.__str__(extended=True), rot_rate_orig.__str__(extended=True))

# 2. Pre-Processing

In [ ]:
rot_rate = rot_rate_orig.copy()
acc = acc_orig.copy()

freqmin = 0.08
freqmax = 50
df = 100

# filter out low frequencies
rot_rate.detrend('constant').filter('highpass', freq = freqmin, zerophase = True)
acc.detrend('constant').filter('highpass', freq = freqmin, zerophase = True)

# filter out high frequencies (25Hz) and resample to 50 Hz
rot_rate.filter('lowpass', freq = freqmax, zerophase = True).resample(sampling_rate=df)
acc.filter('lowpass', freq = freqmax, zerophase = True).resample(sampling_rate=df)

# integrate to nrad and m
disp = acc.copy().integrate().detrend('simple').integrate().filter('highpass', freq = freqmin, zerophase = True)
vel = acc.copy().integrate().detrend('simple').filter('highpass', freq = freqmin, zerophase = True)
angle = rot_rate.integrate().filter('highpass', freq = freqmin, zerophase = True)

print(disp[0].stats, angle[0].stats)

# 3. Plotting Timeseries

In [ ]:
data_t = acc.copy() # vel
unit_t = 'm/s/s'
data_r = rot_rate.copy() # rot_rate
unit_r = 'nrad/s' # the rotation data is already in nrad


tstart1 = obspy.UTCDateTime('2023-06-23T06:40:54')
tend1 = obspy.UTCDateTime('2023-06-23T06:43:04.700Z')
tstart2 = obspy.UTCDateTime('2023-06-23T07:05:33.000000Z')
tend2 = obspy.UTCDateTime('2023-06-23T07:07:48.400Z')

tstart = tstart2
tend = tend2
data_t = data_t.slice(starttime=tstart, endtime=tend)
data_r = data_r.slice(starttime=tstart, endtime=tend)

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(14,8), sharex=True)
plt.subplots_adjust(hspace=0, wspace=0.2)
# plot rotations
for ax, ch in zip(axs[:,0], ['Y','Z']):
    ax.plot(data_r.select(channel='DJ%s' %ch)[0].times('matplotlib'),data_r.select(channel='DJ%s' %ch)[0].data, color='red')
    ax.set_ylabel(ch+' ['+ unit_r + '] ')
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
# plot translations
for ax, ch in zip(axs[:,1], ['Y', 'Z']):
    ax.plot(data_t.select(channel='DN%s' %ch)[0].times('matplotlib'),data_t.select(channel='DN%s' %ch)[0].data, color='black')
    ax.set_ylabel(ch+' ['+ unit_t + '] ')
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
    
axs[1,0].set_xlabel('Time')
axs[1,1].set_xlabel('Time')

# Adjust format as needed
fig.autofmt_xdate() 

plt.show()

# 4. Plot Frequency domain

In [ ]:
data_t = acc.copy() # vel
unit_t = 'm/s/s'
data_r = rot_rate.copy() # rot_rate
unit_r = 'nrad/s' # the rotation data is already in nrad

# slice data
tstart = tstart2
tend = tend2
data_t = data_t.slice(starttime=tstart, endtime=tend)
data_r = data_r.slice(starttime=tstart, endtime=tend)

dt = data_t[0].stats.delta
scale_length = 2


fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(14,8), sharex=True)
plt.subplots_adjust(hspace=0, wspace=0.2)
# plot rotations
length = int(len(data_r.select(channel='DJX')[0].data)/scale_length)
for ax, ch in zip(axs[:,0], ['Y','Z']):
    f_r, P_r = signal.welch(data_r.select(channel='DJ%s' % ch)[0].data, 1 / dt, nperseg=length)
    ax.loglog(f_r,P_r, color='red')
    ax.set_ylabel(ch+' ['+ unit_r + '] ')
    ax.set_ylim(top=2e-6, bottom=1e-14)
    ax.grid(which='major', axis='both')
# plot translations
length = int(len(data_t.select(channel='DNX')[0].data)/scale_length)
for ax, ch in zip(axs[:,1], ['Y','Z']):
    f_t, P_t = signal.welch(data_t.select(channel='DN%s' % ch)[0].data, 1 / dt, nperseg=length)
    ax.loglog(f_t,P_t, color='black')
    ax.set_ylabel(ch+' ['+ unit_t + '] ')
    ax.set_ylim(top=2e0, bottom=1e-7)
    ax.grid(which='major', axis='both')
    
axs[1,0].set_xlabel('Frequency [Hz]')
axs[1,1].set_xlabel('Frequency [Hz]')

axs[0,0].set_xlim(left=0.1, right=10)

  # Adjust format as needed
fig.autofmt_xdate() 

plt.show()


# 5. Prepare for Peak Picking
In this section we prepare for the peak picking. 
1. identify the approximate modal frequencies
2. define a band around these frequencies
3. identify the main motion direction
4. check that the frequencies and the bands are done correctly

For number 3 it is important to pick a direction that is clearly above the sensor noise, so that we don't end up picking noise.

In [ ]:
data_t = acc.copy() # vel
unit_t = 'm/s/s'
data_r = rot_rate.copy() # rot_rate
unit_r = 'nrad/s' # the rotation data is already in nrad

# slice data
tstart = tstart2
tend = tend2
data_t = data_t.slice(starttime=tstart, endtime=tend)
data_r = data_r.slice(starttime=tstart, endtime=tend)

dt = data_t[0].stats.delta

 # 1. identify the approximate modal frequencies
f_oI = [2.5]

# 2. define a band around these frequencies
f_band = [[1.5, 3.9]]

# 3. identify the main motion direction
main_ch_allf = ['DNY']


ch = ['DNY','DNZ']
chr = ['DJY','DJZ']
axis = ['Y','Z']

scale_length = 2

# 4. check that the frequencies and the bands are done correctly
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(14,8), sharex=True)
plt.subplots_adjust(hspace=0, wspace=0.2)
# plot rotations
length = int(len(data_r.select(channel='DJX')[0].data)/scale_length)
for ax, ch in zip(axs[:,0], ['Y','Z']):
    f_r, P_r = signal.welch(data_r.select(channel='DJ%s' % ch)[0].data, 1 / dt, nperseg=length)
    ax.loglog(f_r,P_r, color='red')
    ax.set_ylabel(ch+' ['+ unit_r + '] ')
    ax.set_ylim(top=2e-6, bottom=1e-14)
    ax.grid(which='major', axis='both')
    for f in f_oI:
        ax.axvline(f, color='grey')
    for fmin, fmax in f_band:
        ax.axvline(fmin, color='lightblue')
        ax.axvline(fmax, color='darkblue')
# plot translations
length = int(len(data_t.select(channel='DNX')[0].data)/scale_length)
for ax, ch in zip(axs[:,1], ['Y','Z']):
    f_t, P_t = signal.welch(data_t.select(channel='DN%s' % ch)[0].data, 1 / dt, nperseg=length)
    ax.loglog(f_t,P_t, color='black')
    ax.set_ylabel(ch+' ['+ unit_t + '] ')
    ax.set_ylim(top=2e-2, bottom=1e-15)
    ax.grid(which='major', axis='both')
    for f in f_oI:
        ax.axvline(f, color='grey')
    for fmin, fmax in f_band:
        ax.axvline(fmin, color='lightblue')
        ax.axvline(fmax, color='darkblue')
    
axs[1,0].set_xlabel('Frequency [Hz]')
axs[1,1].set_xlabel('Frequency [Hz]')

axs[0,0].set_xlim(left=0.1, right=10)

  # Adjust format as needed
fig.autofmt_xdate() 

plt.show()

# 6. Frequency Picking on each axis

## 6.1 Calculations

In [ ]:
freqmin = 0.08
freqmax = 20
df = 100

# filter out low frequencies
rot_rate.detrend('constant').filter('highpass', freq = freqmin, zerophase = True)
acc.detrend('constant').filter('highpass', freq = freqmin, zerophase = True)

# filter out high frequencies (50Hz) and resample to 100 Hz
rot_rate.filter('lowpass', freq = freqmax, zerophase = True).resample(sampling_rate=df)
acc.filter('lowpass', freq = freqmax, zerophase = True).resample(sampling_rate=df)

# integrate to  m/s
vel = acc.copy().integrate().detrend('simple').filter('highpass', freq = freqmin, zerophase = True)

data_t = acc # vel
unit_t = 'm/s/s'
data_r = rot_rate # rot_rate
unit_r = 'nrad/s' # the rotation data is already in nrad


# slice data


save_matrix_each = []

for tstart, tend in zip([tstart1,tstart2], [tend1,tend2]):
    data_t_slice = data_t.copy().slice(starttime=tstart, endtime=tend)
    data_r_slice = data_r.copy().slice(starttime=tstart, endtime=tend)
    
    dt = data_t_slice[0].stats.delta
    scale_length = 30
    
    
    ch = ['DNX','DNY','DNZ']
    chr = ['DJX','DJY','DJZ']
    axis = ['X','Y','Z']
    
    for n in range(len(f_oI)):
        for i in range(3):
            ch_t = ch[i]
            ch_r = chr[i]
            freqmin, freqmax = f_band[n][0], f_band[n][1]   # Frequency band limits
            
            #######################################################################################
            # Translation:
            # 1. calculate power spectral density using signal.welch() to get amplitudes "P' and frequencies "f".
            length = int(len(data_t_slice.select(channel=ch_t)[0].data)/scale_length)
            f_t, P_t = signal.welch(data_t_slice.select(channel=ch_t)[0].data, 1 / dt, nperseg=length)
            # 2. get indexes of the frequencies defined in "f_band".
            index_fmin = numpy.abs(f_t - freqmin).argmin()
            index_fmax = numpy.abs(f_t - freqmax).argmin()
            
            # Identify peak amplitude within range
            # 3. peak amplitude and relative index
            single_peak_ampl_t = numpy.max(P_t[index_fmin:index_fmax+1])
            index_peak_ampl_t = numpy.where(P_t[index_fmin:index_fmax+1] == single_peak_ampl_t)[0][0]
            # peak index:
            new_middle = index_fmin + index_peak_ampl_t
        
            # peak amplitude of top 3 picks
            peak_ampl_t = numpy.mean(P_t[new_middle - 1:new_middle + 2])
            # 4. frequency corresponding to peak amplitude
            peak_f_t = f_t[new_middle]
            
            #######################################################################################
            # Rotation:
            # 1. calculate power spectral density using signal.welch() to get amplitudes "P' and frequencies "f".
            length = int(len(data_r_slice.select(channel=ch_r)[0].data)/scale_length)
            f_r, P_r = signal.welch(data_r_slice.select(channel=ch_r)[0].data, 1 / dt, nperseg=length)
            # 2. get indexes of the frequencies defined in "f_band".
            index_fmin = numpy.abs(f_r - freqmin).argmin()
            index_fmax = numpy.abs(f_r - freqmax).argmin()
        
            # Identify peak amplitude within range
            # 3. peak amplitude and relative index
            single_peak_ampl_r = numpy.max(P_r[index_fmin:index_fmax+1])
            index_peak_ampl_r = numpy.where(P_r[index_fmin:index_fmax+1] == single_peak_ampl_r)[0][0]
            # peak index:
            new_middle = index_fmin + index_peak_ampl_r
        
            # peak amplitude of top 3 picks
            peak_ampl_r = numpy.mean(P_r[new_middle - 1:new_middle + 2])
            # 4. frequency corresponding to peak amplitude
            peak_f_r = f_r[new_middle]
    
            # Format timestamp for output
            start_str = str(data_r_slice[0].stats.starttime)
            newfilename = '' + start_str[0:4] + start_str[5:7] + start_str[8:10] + start_str[11:13] + start_str[14:16] + start_str[17:19] + ''
            # date&time, [x,y,z],        mode,           mean ampl trans, mean ampl rot, f trans, f rot
            #  float   , [E,N,Z],  [1, 2, 3, 4, 5, 6],        float     ,   float      ,   float, float
            #  0,           1,            2,                  3,               4,           5,       6
            save_line = [float(newfilename),axis[i],n,numpy.sqrt(peak_ampl_t*peak_f_t),numpy.sqrt(peak_ampl_r*peak_f_r), peak_f_t, peak_f_r]
            save_matrix_each.append(save_line)
ma_picks_individual = numpy.asarray(save_matrix_each)
print(ma_picks_individual)

## 6.2 Plotting the frequencies

In [ ]:
# 6 modes, 6 frequency estimates

fig, axs = plt.subplots(1,2)
plt.subplots_adjust(wspace=0.1)

# Define translation and rotation channels for each mode

channel_N_t = ['DNY'] * 6
channel_Z_t = ['DNZ'] * 6

channel_N_r = ['DJY'] * 6
channel_Z_r = ['DJZ'] * 6

# Extract frequency values from ma_picks_individual dataset

N_t = ma_picks_individual[ma_picks_individual[:,1] == 'Y'][:,5].astype(float)
Z_t = ma_picks_individual[ma_picks_individual[:,1] == 'Z'][:,5].astype(float)

N_r = ma_picks_individual[ma_picks_individual[:,1] == 'Y'][:,6].astype(float)
Z_r = ma_picks_individual[ma_picks_individual[:,1] == 'Z'][:,6].astype(float)

# Scatter plot of frequency estimates
for freq, channel in zip([N_t, Z_t, N_r, Z_r], [channel_N_t, channel_Z_t, channel_N_r, channel_Z_r]):
    for f, ch, ax in zip(freq, channel, axs):
        ax.scatter(f, ch)
        
for ax in axs:
    ax.grid(axis='both')
    ax.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    labelrotation=45,
    grid_linewidth=0.4,
    grid_color='lightgrey')
    
for ax in axs[1:]:
    ax.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,
    labelleft=False,
    grid_linewidth=0.4,
    grid_color='lightgrey')
axs[0].set_title('pre-event')
axs[1].set_title('post-event')

plt.show()

In [ ]:

data_t = acc.copy() # vel
unit_t = '(m/s/s)^2/Hz'
data_r = rot_rate.copy() # rot_rate
unit_r = '(nrad/s)^2/Hz' # the rotation data is already in nrad




fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(11,8), sharex=True)
plt.subplots_adjust(hspace=0, wspace=0.2)

for start, end, color, label in zip([tstart1, tstart2], [tend1, tend2], [['red', 'black'], ['indianred', 'grey']], ['pre-event', 'post-event']):
    # slice data
    data_t_s = data_t.copy().slice(starttime=start, endtime=end)
    data_r_s = data_r.copy().slice(starttime=start, endtime=end)
    
    dt = data_t_s[0].stats.delta
    scale_length = 2
    
    # plot rotations
    length = int(len(data_r_s.select(channel='DJX')[0].data)/scale_length)
    for ax, ch in zip(axs[:,0], ['Y','Z']):
        f_r, P_r = signal.welch(data_r_s.select(channel='DJ%s' % ch)[0].data, 1 / dt, nperseg=length)
        ax.loglog(f_r,P_r, color=color[0], label=label)
        ax.set_ylabel(ch+' ['+ unit_r + '] ')
        ax.set_ylim(top=2e-6, bottom=1e-14)
    # plot translations
    length = int(len(data_t_s.select(channel='DNX')[0].data)/scale_length)
    for ax, ch in zip(axs[:,1], ['Y','Z']):
        f_t, P_t = signal.welch(data_t_s.select(channel='DN%s' % ch)[0].data, 1 / dt, nperseg=length)
        ax.loglog(f_t,P_t, color=color[1], label=label)
        ax.set_ylabel(ch+' ['+ unit_t + '] ')
        ax.set_ylim(top=2e0, bottom=1e-7)
        

for ax in axs.flatten():
    ax.grid(which='major', axis='both')
    
axs[1,0].set_xlabel('Frequency [Hz]')
axs[1,1].set_xlabel('Frequency [Hz]')
axs[0,0].legend()
axs[0,1].legend()
axs[0,0].set_xlim(left=0.1, right=10)

  # Adjust format as needed
fig.autofmt_xdate() 

plt.show()
